# Rapport du projet final : reconnaisance de signature - William GALBY

## Contexte :

Depuis 3 mois, le campus Ynov de Nantes, dans un souci d’écologie mais surtout dans un but de gain de temps, expérimente une nouvelle façon de dématérialiser les feuilles d’émargement des cours qu’il dispense : le progiciel ‘’ JeSuisEnCours ‘’ de la société JDS Conseil. Bientôt éprouvé et approuvé par l’administration et ses utilisateurs finaux (front office), ce système permettra également d’automatiser bien d’autres tâches rébarbatives liées aux absences par rapport aux plages horaires prédéfinit  pour chaque étudiants.

## Problematique :

Les avantages ne sont pas à démontrer tellement ils sont légions. A contrario, les inconvénients se font plus rare et se découvre à l’utilisation au quotidien. En effet, en plus des inconvénients pour les étudiants, à savoir, une feuille d’émargement fixe (dans le mur à l'extérieur de la salle) et des horaires très stricts, ce système a également un défaut pour les utilisateurs côté "back office" (l’administration du campus) : Les tablettes mettent un élève présent dès le moindre trait de crayon virtuel. Il n’y a aucune vérification de l’étudiant qui signe ainsi que de la véracité de la signature.

Ce qui fait que les étudiants ne prennent pas au sérieux cette nouvelle technologie et des débordement se feront vite ressentir (signature à la place des camarades, dessins ou texte à la place de signature, etc.)

## Ma solution :

Je propose pour résoudre ce problème, d’apprendre à partir d’un premier dataset de signatures manuscrite puis d’apprend après chaque nouvelle signature la véracité de celle-ci.

Ensuite, je laisserai la possibilité à l'administration d'Ynov Nantes de saisir le degré de précision de la véracité avant de signaler que la signature est fausse.

### Dans la théorie :

J’aurais utilisé les API que le progiciel fournit, à savoir L’API GetSignatureList

La documentation dit : "Celon le site internet, elle permet de récupérer la liste des présences en cours et des images PNG des signatures. Cette API permet également de mettre à jour en quasi temps réel les systèmes d’appels intégrés dans les logiciels métiers des OF et de déclencher des alertes pour les apprenants notés absents à l’heure de fermeture de la tranche horaire de signature autorisée.

<img src="..\API.jpg">

Une URL unique par signature recueillie permet aussi de réaliser une interface vers l’image de la signature, soit par récupération HTTPS de l’image en question, soit en intégrant un lien URL dans le logiciel métier, autorisant un accès direct à la signature unitaire d’un cours pour un stagiaire ou un formateur." - Source : https://jesuisencours.com/api

### Dans la pratique :

Après un exposé des points ci-dessus à l’administration, ils leurs est apparu un vrai atout. Néanmoins, après questionnement de ces derniers comme le montre l'impression d'écran ci-dessous, ils ne sont pas prêt à implémenter cette fonctionnalité directement en production par peur faire défaillir le systeme et ses avantages cité au-dessus (la signature est au cœur de ce système), ils m’ont demandé de leur creer un programme fonctionnel dans un premier temps. 

#### 1ère tentative :
<img src="..\IMG-20170306-WA0003.jpg">

#### 2ème tentative :
<img src="..\IMG-20170306-WA0002.jpg">

Je vais donc simuler un environement qui ressemble le plus à la réalité de l'environement en production. En effet, sans possibilité d'appel à l'api jesuisencours, il est inutile de faire des tests sur des images puisque je n'aurais jamais les images des signatures. C'est pour cela qu'il me faut trouver un moyen de me rapprocher le plus possible du contexte d'une signature sur tablette comme le fait jesuisencours (le remplacer entièrement en quelques sortes).

Pour cela, je vais utiliser l'algorithme one-dollar (le meilleur algo de reconnaisance d'écriture d'après les resultats de mes tests, il a été créé par Jacob O. Wobbrock, University of Washington, Andrew D. Wilson, Microsoft Research, Yang Li, University of Washington : tous travaillant chez Google), ainsi que le langage python avec de surcroit : la librairie pygame.

Je ne vais donc pas ici créer un dataset d’images de signatures fait avec un logiciel tier (comme paint.net par exemple) pour simuler une reception de l'api jesuisencours, mais bien créer un nuage de points que je vais enregistrer directement auprès de l’utilisateur. Une fois récupérer, mon programme stockera ces nuages de points pour servir de base de données d’apprentissage. Ensuite il faudra renseigner quelques variables gloables permettant de personaliser le programme :
- Nom
- Prénom
- Précision > Ce paramètre que va indiquer l'administration sera le seuille le plus bas à ne pas dépasser (entre 0% et 200%) sachant que plus le pourcentage est petit, plus la signature est véridique

Dès le lancement de la partie du programme qui reconnait les signatures, une nouvelle fenêtre pygame s’ouvre pour taper la fameuse signature à vérifier.
Schéma récapitulatif étape par étape comme suit :

<img src="..\APImaj.jpg">

Concernant la sortie de mon programme nous aurons donc un tableau de concordance avec le nom des étudiants ainsi que leur score de véracités de signatures sachant que plus on se rapproche de 1 et plus la signature est correct. Nous aurons  donc un tableau avec les colonnes suivantes :
- Nom : de l'étudiants STRING
- Prénom : de l'étudiants STRING
- Signature du dataset : Dataset de nuage de point de l'étudiant courant LISTE 2D
- Signature à tester : Un nuage de point qui correspond à la signature à vérifier LISTE 1D
- Score en % : Score de la véracité de la signature (celle-ci dessus par rapport au dataset) FLOAT en pourcent sachant que plus c'est un petit chiffre, plus la véracité est bonne
- Précision : Paramètre qui détermine le pourcentage d'acceptabilité de la véracité de la signature (va se comparer à la moyenne des résultats des scores de chaque signature du dataset) FLOAT de O à 1
- MOYENNE : Total > La moyenne des scores de chaque signature du dataset de nuage de point
- VALIDATION : Résultat final > Par rapport à la moyenne et à la précision indiqué, détermine si la signature est correct ou non.

Toujours dans l'optique de remplacer le progiciel jesuisencours mais également dans le but de creer un reel programme de machine leaning, ces résutlats se stockent dans un CSV qui me sert de base de donnée. A chaque lancement complet du programme, tout s'enregistre à la suite pour que les résultats soient de plus en plus précit, signature après signature

## Conclusion

Il y a deux seuls petits points négatifs du programme : Lorsqu'un utlisateur rentre son dataset de signatures, il n'y a pas de vérification donc si il en loupe une, il doit relancer le programme (les éventuelles données résiduelles creer par l'utilisateur pourraient biaiser les résultats). Le deuxième point négatif est que le programme est fait pour traiter une personne à la fois, si une personne le lance et passe la mains à la suivante sans sauvegarder ses résultats, sont CSV se verra écrasé. Si ce n'est ces deux points, les résultats sont très concluant, le tout est très fonctionnel. C'est pour cette raison que je pense que mon programme vaut quand même 16/20.